In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kritanjalijain/amazon-reviews")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'amazon-reviews' dataset.
Path to dataset files: /kaggle/input/amazon-reviews


In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
df_train_original = pd.read_csv(f"{path}/train.csv" , header=None)
df_test_original = pd.read_csv(f'{path}/test.csv' , header = None)

In [9]:
df_train_original.columns = ['polarity', 'title', 'text']
df_test_original.columns = ['polarity', 'title', 'text']

In [41]:
from sklearn.model_selection import train_test_split

# Assuming df_train has columns like: ['text', 'polarity', ...]
df_train, _ = train_test_split(
    df_train_original,
    train_size=1_000_000,
    stratify=df_train_original['polarity'],  # keeps label balance
    random_state=42
)

df_test, _ = train_test_split(
    df_test_original,
    train_size=100_000,
    stratify=df_test_original['polarity'],  # keeps label balance
    random_state=42
)

print(df_train.shape)

print(df_test.shape)

(1000000, 3)
(100000, 3)


In [42]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000000 entries, 2594248 to 592070
Data columns (total 3 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   polarity  1000000 non-null  int64 
 1   title     999932 non-null   object
 2   text      1000000 non-null  object
dtypes: int64(1), object(2)
memory usage: 30.5+ MB


In [43]:
df_train.head()

,polarity,title,text
2594248,2,Great Resource!,As both a teacher and an aunt of three childre...
494080,1,Not quite,I purchased this system to use at a summer cot...
1709691,1,NOT recommended,I purchased this and after only about 1 month ...
1821812,1,Trying too hard!!,This was a very painful book to read and equiv...
2351691,1,Where's the steam?,This vaporizer worked great for the first 2 da...


In [44]:
# polarity column description
# 1 == Negative
# 2 == Positive
df_train['polarity'].unique()

array([2, 1])

In [45]:
df_train['polarity'].value_counts()

,count
polarity,
2,500000
1,500000


In [46]:
# LOWER CASE THE DATA OF COLUMN 'title'

df_train['title'] = df_train['title'].str.lower()

In [47]:
# LOWER CASE THE DATA OF COLUMN 'text'

df_train['text'] = df_train['text'].str.lower()

In [48]:
df_train['title'].sample(10)

,title
3313710,too small to be of use
1351664,sleeping beauty ruminations
2959300,it's about time!
2213412,no sd slot
3455808,disjointed and dull
1409055,every hazel fan needs this
2413385,usealot
1134986,ripped off!
261782,signal drop-off common w/this unit
532260,done and wish i had the time back


In [49]:
print("shape of df_train: ",df_train.shape)
print("shape of df_test: ",df_test.shape)

shape of df_train:  (1000000, 3)
shape of df_test:  (100000, 3)


In [50]:
# nan values in df_train
df_train.isna().sum()

,0
polarity,0
title,68
text,0


In [51]:
#  nan values in df_test
df_test.isna().sum()

,0
polarity,0
title,2
text,0


In [52]:
print(df_train.duplicated().sum())
print(df_test.duplicated().sum())

34
0


In [53]:
df_train = df_train.drop_duplicates()
df_test = df_test.drop_duplicates()

In [54]:
# drop nan values
df_train = df_train.dropna()
df_test = df_test.dropna()

In [55]:
#  droping title as i think it is not needed for now

df_train = df_train.drop(['title'], axis=1)
df_test = df_test.drop(['title'], axis=1)

In [56]:
import string
translator = str.maketrans('','',string.punctuation)
digit = str.maketrans('','',string.digits)

In [57]:
# removing punctuations from title
# df_train['title'] = df_train['title'].apply(lambda x: x.translate(translator))

In [58]:
# removing punctuations from text
df_train['text'] = df_train['text'].apply(lambda x: x.translate(translator))

In [59]:
# removing punctuations from text
df_test['text'] = df_test['text'].apply(lambda x: x.translate(translator))

In [60]:
df_train.sample(10)

,polarity,text
2076067,2,if you are a chap like myself who enjoy movies...
2061933,2,hes simply the best filmmaker ever for smart g...
2911882,1,very cheap construction flimsy plastic the wir...
3199331,1,this battery came bulged to even get it into t...
1329023,2,others will give a better low down on the stor...
2127243,2,this series is very strong and full of explici...
1619300,1,got a new keurig special edition machine after...
1410326,2,the carnival bizarre is a truly awesome album ...
1264453,1,not worth the purchase price would love to hea...
2722196,1,call me a sour old cow but looking at the revi...


In [61]:
# removing the numbers from the "title"
# df_train['title'] = df_train['title'].apply(lambda x: x.translate(digit))

In [62]:
# removing the numbers from the "text"
df_train['text'] = df_train['text'].apply(lambda x: x.translate(digit))

In [63]:
# removing the numbers from the "text"
df_test['text'] = df_test['text'].apply(lambda x: x.translate(digit))

In [64]:
df_train.sample(10)

,polarity,text
595823,2,we tried the ghirardelli hot cocoa with our ke...
2222382,1,plugged it into the usb and downloaded songs i...
3313254,2,these collars are the best available they are ...
1578419,1,the hanes comfortsoft waistband woven boxers a...
2763439,2,this is definitely a great guide to review if ...
997538,2,this movie is a nice combination of a touch of...
3321524,2,the patronizing review of charles wright revea...
1635279,1,first off this album is bad nd ludacris has no...
2157760,2,i bought these because they are bigger in surf...
2276808,2,ive gotten halfway through this book but had t...


In [65]:
# removing extra spaces just in case
# df_train['title'] = df_train['title'].str.strip()
df_train['text'] = df_train['text'].str.strip()


In [66]:

df_test['text'] = df_test['text'].str.strip()

In [67]:
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [68]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_r

True

In [69]:
stopword = set(stopwords.words('english'))
stemmer = PorterStemmer()

In [70]:
from tqdm import tqdm
tqdm.pandas()

In [71]:
# removing stop words and also applying the tokenizations

def tokenization_and_stopwords_removal(text):
    tokenized_text = word_tokenize(text)
    texts = [word for word in tokenized_text if word.lower() not in stopword ]
    stemmed_text = [stemmer.stem(word) for word in texts]


    return " ".join(stemmed_text)

df_train['text'] = df_train['text'].progress_apply(tokenization_and_stopwords_removal)

100%|██████████| 999898/999898 [18:31<00:00, 899.69it/s]


In [72]:
df_test['text'] = df_test['text'].progress_apply(tokenization_and_stopwords_removal)

100%|██████████| 99998/99998 [01:50<00:00, 904.61it/s]


In [73]:

df_train.sample(10)

,polarity,text
222147,2,ever play sport time stood still experienc flo...
515281,1,sit entir film felt longer state run time hard...
1937056,2,firstli hous haunt hill actual pretti good mov...
201718,2,bought replac vh tapeand realli enjoy itth act...
2275405,2,fob doin thang theyr pretti good favorit favor...
2262791,1,dont buy album unless want listen grate second...
2538588,2,bought week ago got singapor router work inten...
35817,2,actual didnt use book much surround almost eng...
2961788,1,silli book pick hope someth littl realist inst...
1274099,1,far perform hope local free hd channel antenna...


In [74]:
print("df_train['polarity] ",  df_train['polarity'].value_counts())
print("")
print("df_test['polarity] ",  df_test['polarity'].value_counts())


df_train['polarity]  polarity
2    499972
1    499926
Name: count, dtype: int64

df_test['polarity]  polarity
2    50000
1    49998
Name: count, dtype: int64


In [75]:
print(f"df_test is null or not\n\n {df_test.isnull().sum()}")
print('')
print(f"df_train is null or not\n\n {df_train.isnull().sum()}")

df_test is null or not

 polarity    0
text        0
dtype: int64

df_train is null or not

 polarity    0
text        0
dtype: int64


In [76]:
# now negative = 0, positive = 1 for simplicity

df_train['polarity'] = df_train['polarity'].map({2:1, 1:0})
df_test['polarity'] = df_test['polarity'].map({2:1,1:0})

In [77]:
X_train = df_train['text'].astype('str')
Y_train = df_train['polarity']
X_val = df_test['text'].astype('str')
Y_val = df_test['polarity']

In [78]:
!pip install gensim tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 18.6 MB/s eta 0:00:00


In [ ]:
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer

vocab_size = 100000
tokenizer = Tokenizer(num_words=vocab_size , oov_token="<OOV>")

word_index = tokenizer.word_index
tokenizer.fit_on_texts(X_train)


w2v_model = Word2Vec(sentences=[t.split() for t in X_train],
                     vector_size=300,
                     window=3,
                     min_count=5,
                     workers=8
                     )



embedding_dim = 300
embedding_matrix = np.zeros((vocab_size , embedding_dim))

for word , i in word_index.items():
    if i < vocab_size:
        if word in w2v_model.wv:
            embedding_matrix[i] = w2v_model.wv[word]

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


Sequences = tokenizer.texts_to_sequences(X_train)
val_sequences = tokenizer.texts_to_sequences(X_val)



maxlen = 800
X_train = pad_sequences(sequences=Sequences, maxlen=maxlen, padding='post')

X_val = pad_sequences(sequences=val_sequences, maxlen=maxlen, padding='post')

In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM , Embedding , Bidirectional , Dense , Dropout

model = Sequential([
    Embedding(input_dim=vocab_size,
              output_dim=300,
              weights=[embedding_matrix],
              input_length=maxlen,
              trainable=True
              ),

              Bidirectional(LSTM(128, return_sequences=True, dropout = 0.5, recurrent_dropout=0.3)),
              Bidirectional(LSTM(64, dropout = 0.5, recurrent_dropout=0.3)),

              Dropout(0.5),
              Dense(64, activation="relu"),
              Dense(1,activation="sigmoid")
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(X_train , Y_train,
          validation_data=(X_val, Y_val),
          batch_size=1024,
          epochs=3,
          verbose=1)

In [ ]:
loss, accuracy = model.evaluate(X_val, Y_val, verbose=1)
print(f"Validation Accuracy: {accuracy:.4f}")


In [ ]:


plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
model.save("sentiment_bilstm_word2vec.h5")
